학습된 하나의 모델을 사용하는 것보다 학습된 여러개의 모델을 사용하면 보다 정확한 예측을 기대할 수 있다.

voting은 여러 예측 모델을 두고 각 모델이 voting하여 최종 결정을 하며,\
bagging은 하나의 모델을 여러개의 데이터 샘플링하여 학습하고 voting하여 최종 결정한다.\
각각 데이터 샘플링하는 방법은 부트스트래핑 분할방식이라고 한다.\
하드보팅은 최종 결정에서 여러개의 모델이 선택한 결정을 따르며, 소프트 보팅은 각 모델이 게인값을 갖고 최종 결정을 한다.

random forest는 bagging의 대표적인 방법이며,\
부트스트래핑으로 샘플링한 데이터마다 예측한 결과를 소프트 보팅하여 최종 예측 결정을 한다.\
Random forest는 앙상블 방법 중 속도가 빠르고 높은 성능을 보여준다.\
Human Activity Recognition Data를 Random forest 기법을 활용하여 분석해본다.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


/tmp/ipykernel_13487/2609744967.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
url =  'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/features.txt'
feature_name_df = pd.read_csv(url, sep='\s+', header=None, names=['column_index', 'column_name'])
feature_name = feature_name_df.iloc[:,1].values.tolist()

In [3]:
x_train_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/train/X_train.txt'
x_test_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/test/X_test.txt'
y_train_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/train/y_train.txt'
y_test_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/test/y_test.txt'

x_train = pd.read_csv(x_train_url, sep='\s+', header=None)
x_test = pd.read_csv(x_test_url, sep='\s+', header=None)
y_train = pd.read_csv(y_train_url, sep='\s+', header=None, names=['action'])
y_test = pd.read_csv(y_test_url, sep='\s+', header=None, names=['action'])

x_train.columns = feature_name
x_test.columns = feature_name

In [4]:
dt_clf = DecisionTreeClassifier(max_depth=5, random_state=5)
dt_clf.fit(x_train, y_train)
pred = dt_clf.predict(x_test)
accuracy_score(y_test, pred)

0.839497794367153

In [6]:
from sklearn.model_selection import GridSearchCV

params = {'max_depth':list(range(6,18,2))}
grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, return_train_score=True)
grid_cv.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=5, random_state=5),
             param_grid={'max_depth': [6, 8, 10, 12, 14, 16]},
             return_train_score=True, scoring='accuracy')

In [7]:
cv_result_df = pd.DataFrame(grid_cv.cv_results_)
cv_result_df[['param_max_depth', 'mean_test_score', 'mean_train_score']]

,param_max_depth,mean_test_score,mean_train_score
0,6,0.843989,0.944879
1,8,0.848078,0.982692
2,10,0.847399,0.993403
3,12,0.843727,0.997212
4,14,0.849304,0.998776
5,16,0.844543,0.999660


In [8]:
max_depth = list(range(6,17))

for depth in max_depth :
    dt_clf = DecisionTreeClassifier(max_depth=depth, random_state=5)
    dt_clf.fit(x_train, y_train)
    pred = dt_clf.predict(x_test)
    accuracy = accuracy_score(y_test, pred)
    print('Max_Depth = ', depth, ', Accuracy = ', accuracy)

Max_Depth =  6 , Accuracy =  0.8551068883610451
Max_Depth =  7 , Accuracy =  0.8618934509670851
Max_Depth =  8 , Accuracy =  0.8720732948761453
Max_Depth =  9 , Accuracy =  0.8717339667458432
Max_Depth =  10 , Accuracy =  0.8703766542246352
Max_Depth =  11 , Accuracy =  0.8686800135731252
Max_Depth =  12 , Accuracy =  0.8656260604004072
Max_Depth =  13 , Accuracy =  0.8642687478791992
Max_Depth =  14 , Accuracy =  0.8571428571428571
Max_Depth =  15 , Accuracy =  0.8578215134034611
Max_Depth =  16 , Accuracy =  0.8540889039701391


In [9]:
best_df_clf = grid_cv.best_estimator_
pred1 = best_df_clf.predict(x_test)
accuracy_score(y_test, pred1)

0.8571428571428571

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

params = {
    'max_depth' : [6, 8, 10],
    'n_estimators' : [50, 100, 200],
    'min_samples_leaf' : [8, 12],
    'min_samples_split' : [8, 12]
}

rf_clf = RandomForestClassifier(random_state=5, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(x_train, y_train)

/home/han/venv/myvenv/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/han/venv/myvenv/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/han/venv/myvenv/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/han/venv/myvenv/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the

GridSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=-1, random_state=5),
             n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10], 'min_samples_leaf': [8, 12],
                         'min_samples_split': [8, 12],
                         'n_estimators': [50, 100, 200]})

In [11]:
cv_results_df = pd.DataFrame(grid_cv.cv_results_)
cv_results_df.columns

target_col = ['rank_test_score', 'mean_test_score', 'param_n_estimators', 'param_max_depth']
cv_result_df[target_col].sort_values('rank_test_score').head()

KeyError: "['param_n_estimators'] not in index"